In [8]:

%matplotlib widget

import cell_model_base as cs_model
import detect_peaks as dp
import numpy as np
import matplotlib.pyplot as plt
import spatial_metrics_base as pl
import os
import helper_functions as hf
import scipy.stats as stats

path = os.path.abspath(os.path.dirname(pl.__file__))



In [25]:

mean_video_srate = 100.
total_Time = 500
_lambda = 10

x_coordinates,y_coordinates,speed,timevector = cs_model.generate_randomWalk2(input_srate = mean_video_srate,input_total_Time = total_Time,heading_srate = 10., speed_srate = 5., rho1  = 1,sigma = 0.02,mu_e  = 0.,smooth_coeff = 0.5)
All_arrival_time,I_timestamps = cs_model.generate_arrivals(_lambda,total_Time)

# modulated_timestamps = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 5)

# modulated_timestamps1 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 5)
# modulated_timestamps2 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 10, s = 5)
# modulated_timestamps3 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 90, s = 5)
# modulated_timestamps4 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 90, s = 5)
# modulated_timestamps5 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 10, s = 5)
# modulated_timestamps = np.unique(np.concatenate([modulated_timestamps1,modulated_timestamps2,modulated_timestamps3,modulated_timestamps4,modulated_timestamps5]))

modulated_timestamps1 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50, y_center = 90, s = 5)
modulated_timestamps2 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10, y_center = 10, s = 5)
modulated_timestamps3 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90, y_center = 10, s = 5)
modulated_timestamps = np.unique(np.concatenate([modulated_timestamps1,modulated_timestamps2,modulated_timestamps3]))


modulated_timestamps = np.sort(np.random.choice(modulated_timestamps,50,replace=False))
# modulated_timestamps = np.sort(np.random.choice(I_timestamps,50,replace=False))

calcium_imag,timevector = cs_model.generate_CalciumSignal(modulated_timestamps,noise_level = 0.01, b = 5.)
calcium_events = modulated_timestamps.shape[0]
I_peaks = dp.detect_peaks(calcium_imag,mpd=0.5*mean_video_srate,mph=1.*np.nanstd(calcium_imag))


place_cell = pl.PlaceCell(RatSession='Sst',day = 1, ch = 1, dataset = 'Exploration',mean_video_srate = 100.,mintimespent = 0.1,
                          minvisits = 1, speed_threshold = 2.5, nbins_cal = 10, num_surrogates = 200,num_cores = 20, nbins_pos_x = 10, 
                          nbins_pos_y = 10, placefield_nbins_pos_x = 50, placefield_nbins_pos_y = 50,saving = False,saving_string='CI')



In [21]:
 Les Mills

/home/apolo/Documents/DataAnalysis/Projects/CalciumSignalAnalysis/spatial_metrics_base.py:196: RuntimeWarning: Mean of empty slice
  calcium_mean_occupancy[yy,xx] = np.nanmean(mean_calcium_to_behavior_speed[np.logical_and(check_x_ocuppancy,check_y_ocuppancy)])


File not saved!


In [22]:

PlaceCellObject.keys()


dict_keys(['signalMap', 'place_field', 'place_field_smoothed', 'ocuppancyMap', 'visitsMap', 'x_grid', 'y_grid', 'x_center_bins', 'y_center_bins', 'numb_events', 'events_index', 'mutualInfo_original', 'mutualInfo_zscored', 'mutualInfo_permutation', 'sparsity', 'gridness_permutation', 'gridness_original', 'gridness_zscored'])

In [23]:

plt.figure()
plt.hist(PlaceCellObject['gridness_permutation'],20)
plt.plot([PlaceCellObject['gridness_original'],PlaceCellObject['gridness_original']],[0,25],'r--',linewidth=2)
plt.title(PlaceCellObject['gridness_zscored'])
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:

plt.figure()
cnt = plt.contourf(PlaceCellObject['x_center_bins'],PlaceCellObject['y_center_bins'],PlaceCellObject['place_field_smoothed'],100)
for c in cnt.collections:
    c.set_edgecolor("face")

plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
plt.title(f"MI = {PlaceCellObject['mutualInfo_zscored']:0.3f} ; Events = {calcium_events}")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [192]:

array_output = grid_spatial_autocorrelation(PlaceCellObject['place_field_smoothed'])
array_output


plt.figure()
cnt = plt.contourf(array_output,100)
for c in cnt.collections:
    c.set_edgecolor("face")

plt.colorbar()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [193]:
autoCorrRot = rotate(autoCorr, angle=30, reshape=False)


plt.figure()
cnt = plt.contourf(autoCorrRot,100)
for c in cnt.collections:
    c.set_edgecolor("face")

plt.colorbar()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [201]:
def get_gridness_index(array_output):

    from scipy.ndimage.interpolation import rotate


    array_output_zeroed = np.copy(array_output)
    array_output_zeroed[np.isnan(array_output_zeroed)] = 0
    autoCorr = np.copy(array_output_zeroed)
    da = 3
    angles = list(range(0, 180+da, da))
    crossCorr = []
    # Rotate and compute correlation coefficient
    for angle in angles:
        autoCorrRot = rotate(autoCorr, angle, reshape=False)
        C = np.corrcoef(np.reshape(autoCorr, (1, autoCorr.size)),
            np.reshape(autoCorrRot, (1, autoCorrRot.size)))
        crossCorr.append(C[0, 1])

    max_angles_i = (np.array([30, 90, 150]) / da).astype(int)
    min_angles_i = (np.array([60, 120]) / da).astype(int)

    maxima = np.max(np.array(crossCorr)[max_angles_i])
    minima = np.min(np.array(crossCorr)[min_angles_i])
    G = minima - maxima
    return G

        

In [196]:

plt.figure()
plt.plot(angles,crossCorr,'k')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
import scipy. signal as sig


array_output = sig.correlate2d(PlaceCellObject['place_field_smoothed'],PlaceCellObject['place_field_smoothed'],mode='full')
array_output


plt.figure()
cnt = plt.contourf(array_output,100)
for c in cnt.collections:
    c.set_edgecolor("face")

plt.colorbar()
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [161]:
def grid_spatial_autocorrelation(input_matrix):

    matrix_image = input_matrix.copy()
    kernel = input_matrix.copy()

    [ma,na] = np.shape(matrix_image)
    [mb,nb] = np.shape(kernel)

    mc = np.max([ma+mb-1,ma,mb])
    nc = np.max([na+nb-1,na,nb])

    matrix_output = np.nan*np.zeros([mc,nc])

    i_size = kernel.shape[0]
    j_size = kernel.shape[1]

    kernel_size_i,kernel_size_j = np.shape(kernel)
    matrix_image_size = np.array(np.shape(matrix_image));

    output_matrix_size = matrix_image_size + [2*npad_i-1, 2*npad_j-1];
    work_mat = np.nan * np.zeros(output_matrix_size);
    work_mat[(kernel_size_i):(kernel_size_i+matrix_image_size[0]),(kernel_size_j):(kernel_size_j+matrix_image_size[1])] = matrix_image

    for i in range(np.shape(matrix_output)[0]):
        for j in range(np.shape(matrix_output)[1]):

            win1 = np.arange(i,kernel_size_i+i).astype(int)
            win2 = np.arange(j,kernel_size_j+j).astype(int)

            matrix_sliced = work_mat[win1,:][:,win2]; 
            matrix_sliced_kernel = matrix_sliced*kernel;                                             
            keep = ~np.isnan(matrix_sliced_kernel)

            n = np.sum(keep);

            if n < 20:
                matrix_output[i,j] = np.nan;

            else:

                sum_matrix_kernel_x_lagged = np.sum(matrix_sliced_kernel[keep]);
                sum_matrix_lagged =   np.sum(matrix_sliced[keep]);
                sum_matrix_kernel =   np.sum(kernel[keep]);
                sum_matrix_lagged_2 =  np.sum(matrix_sliced[keep]**2);
                sum_matrix_kernel_2 =  np.sum(kernel[keep]**2);

                matrix_output[i,j] = (n*sum_matrix_kernel_x_lagged - sum_matrix_kernel*sum_matrix_lagged) / (np.sqrt(n*sum_matrix_kernel_2-sum_matrix_kernel**2) * np.sqrt(n*sum_matrix_lagged_2-sum_matrix_lagged**2));

    return matrix_output



In [202]:

array_output = grid_spatial_autocorrelation(PlaceCellObject['place_field_smoothed'])

plt.figure()
cnt = plt.contourf(array_output,100)
for c in cnt.collections:
    c.set_edgecolor("face")

plt.colorbar()
plt.show()
gridness = get_gridness_index(array_output)
print(gridness)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.1743591392198115
